In [1]:
spark_spam = sc.textFile('data/spambase/spambase.data')

In [2]:
spark_spam.take(5)

[u'0,0.64,0.64,0,0.32,0,0,0,0,0,0,0.64,0,0,0,0.32,0,1.29,1.93,0,0.96,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.778,0,0,3.756,61,278,1',
 u'0.21,0.28,0.5,0,0.14,0.28,0.21,0.07,0,0.94,0.21,0.79,0.65,0.21,0.14,0.14,0.07,0.28,3.47,0,1.59,0,0.43,0.43,0,0,0,0,0,0,0,0,0,0,0,0,0.07,0,0,0,0,0,0,0,0,0,0,0,0,0.132,0,0.372,0.18,0.048,5.114,101,1028,1',
 u'0.06,0,0.71,0,1.23,0.19,0.19,0.12,0.64,0.25,0.38,0.45,0.12,0,1.75,0.06,0.06,1.03,1.36,0.32,0.51,0,1.16,0.06,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.06,0,0,0.12,0,0.06,0.06,0,0,0.01,0.143,0,0.276,0.184,0.01,9.821,485,2259,1',
 u'0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0,0,0.31,0,0,3.18,0,0.31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.137,0,0.137,0,0,3.537,40,191,1',
 u'0,0,0,0,0.63,0,0.31,0.63,0.31,0.63,0.31,0.31,0.31,0,0,0.31,0,0,3.18,0,0.31,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.135,0,0.135,0,0,3.537,40,191,1']

In [5]:
import re

In [6]:
names = re.findall('^(.+):', open('data/spambase/spambase.names').read(), re.MULTILINE)
del names[0]
names.append('label')

In [7]:
names

['word_freq_make',
 'word_freq_address',
 'word_freq_all',
 'word_freq_3d',
 'word_freq_our',
 'word_freq_over',
 'word_freq_remove',
 'word_freq_internet',
 'word_freq_order',
 'word_freq_mail',
 'word_freq_receive',
 'word_freq_will',
 'word_freq_people',
 'word_freq_report',
 'word_freq_addresses',
 'word_freq_free',
 'word_freq_business',
 'word_freq_email',
 'word_freq_you',
 'word_freq_credit',
 'word_freq_your',
 'word_freq_font',
 'word_freq_000',
 'word_freq_money',
 'word_freq_hp',
 'word_freq_hpl',
 'word_freq_george',
 'word_freq_650',
 'word_freq_lab',
 'word_freq_labs',
 'word_freq_telnet',
 'word_freq_857',
 'word_freq_data',
 'word_freq_415',
 'word_freq_85',
 'word_freq_technology',
 'word_freq_1999',
 'word_freq_parts',
 'word_freq_pm',
 'word_freq_direct',
 'word_freq_cs',
 'word_freq_meeting',
 'word_freq_original',
 'word_freq_project',
 'word_freq_re',
 'word_freq_edu',
 'word_freq_table',
 'word_freq_conference',
 'char_freq_;',
 'char_freq_(',
 'char_freq_[',
 '

In [144]:
ord(';')

59

In [9]:
def sub_fun(match):
    return 'a' + str(ord(match.group(0)))

In [10]:
re.sub('[^a-z_0-9]', sub_fun, 'char_freq_#')

'char_freq_a35'

In [11]:
spark_names = [re.sub('[^a-z_0-9]', sub_fun, name) for name in names]

In [12]:
from pyspark.sql import Row

In [13]:
def line_to_row(line):
    vals = line.split(',')
    argdict = {name: float(val) for name, val in zip(spark_names, vals)}
    return Row(**argdict)

In [14]:
df = spark.createDataFrame(spark_spam.map(line_to_row))

In [15]:
df.select(df.columns[:5]).show(5)

+--------------------------+--------------------------+------------------------+-------------+-------------+
|capital_run_length_average|capital_run_length_longest|capital_run_length_total|char_freq_a33|char_freq_a35|
+--------------------------+--------------------------+------------------------+-------------+-------------+
|                     3.756|                      61.0|                   278.0|        0.778|          0.0|
|                     5.114|                     101.0|                  1028.0|        0.372|        0.048|
|                     9.821|                     485.0|                  2259.0|        0.276|         0.01|
|                     3.537|                      40.0|                   191.0|        0.137|          0.0|
|                     3.537|                      40.0|                   191.0|        0.135|          0.0|
+--------------------------+--------------------------+------------------------+-------------+-------------+
only showing top 5 

In [17]:
features = ['char_freq_$', 'word_freq_remove', 'char_freq_!']
spark_features = [spark_names[names.index(i)] for i in features]

In [18]:
spark_features

['char_freq_a36', 'word_freq_remove', 'char_freq_a33']

In [19]:
df.select(spark_features).show()

+-------------+----------------+-------------+
|char_freq_a36|word_freq_remove|char_freq_a33|
+-------------+----------------+-------------+
|          0.0|             0.0|        0.778|
|         0.18|            0.21|        0.372|
|        0.184|            0.19|        0.276|
|          0.0|            0.31|        0.137|
|          0.0|            0.31|        0.135|
|          0.0|             0.0|          0.0|
|        0.054|             0.0|        0.164|
|          0.0|             0.0|          0.0|
|        0.203|             0.3|        0.181|
|        0.081|            0.38|        0.244|
|          0.0|            0.96|        0.462|
|          0.0|            0.25|        0.663|
|          0.0|             0.0|        0.786|
|          0.0|             0.9|          0.0|
|          0.0|             0.0|        0.357|
|        0.063|            0.42|        0.572|
|          0.0|             0.0|        0.428|
|         0.37|             0.0|        1.975|
|          0.

In [20]:
from pyspark.ml.feature import RFormula

In [21]:
formula = 'label ~ ' + ' + '.join(spark_features)

In [22]:
selector = RFormula(formula=formula)

In [23]:
X_formula = selector.fit(df).transform(df)

In [24]:
X_formula.select('features', 'label').show(5)

+------------------+-----+
|          features|label|
+------------------+-----+
|   [0.0,0.0,0.778]|  1.0|
| [0.18,0.21,0.372]|  1.0|
|[0.184,0.19,0.276]|  1.0|
|  [0.0,0.31,0.137]|  1.0|
|  [0.0,0.31,0.135]|  1.0|
+------------------+-----+
only showing top 5 rows



In [25]:
from pyspark.ml.feature import VectorAssembler

In [26]:
input_features = spark_names
input_features.remove('label')

In [27]:
X_all = (VectorAssembler(inputCols=input_features, outputCol='features')
         .transform(df).select('features', 'label'))

In [29]:
X_all.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(57,[1,2,4,11,15,...|  1.0|
|(57,[0,1,2,4,5,6,...|  1.0|
|(57,[0,2,4,5,6,7,...|  1.0|
|(57,[4,6,7,8,9,10...|  1.0|
|(57,[4,6,7,8,9,10...|  1.0|
|(57,[4,7,49,54,55...|  1.0|
|(57,[4,9,10,11,15...|  1.0|
|(57,[4,7,49,54,55...|  1.0|
|(57,[0,2,4,6,8,9,...|  1.0|
|(57,[0,1,2,4,5,6,...|  1.0|
|(57,[6,9,10,17,18...|  1.0|
|(57,[2,4,5,6,10,1...|  1.0|
|(57,[1,2,4,11,15,...|  1.0|
|(57,[4,6,9,10,12,...|  1.0|
|(57,[2,4,5,7,9,11...|  1.0|
|(57,[1,2,4,6,9,15...|  1.0|
|(57,[4,18,20,51,5...|  1.0|
|(57,[20,22,51,52,...|  1.0|
|(57,[2,4,6,12,14,...|  1.0|
|(57,[1,4,5,8,11,1...|  1.0|
+--------------------+-----+
only showing top 20 rows



In [31]:
X_all.schema['features'].metadata

{u'ml_attr': {u'attrs': {u'numeric': [{u'idx': 0, u'name': u'word_freq_make'},
    {u'idx': 1, u'name': u'word_freq_address'},
    {u'idx': 2, u'name': u'word_freq_all'},
    {u'idx': 3, u'name': u'word_freq_3d'},
    {u'idx': 4, u'name': u'word_freq_our'},
    {u'idx': 5, u'name': u'word_freq_over'},
    {u'idx': 6, u'name': u'word_freq_remove'},
    {u'idx': 7, u'name': u'word_freq_internet'},
    {u'idx': 8, u'name': u'word_freq_order'},
    {u'idx': 9, u'name': u'word_freq_mail'},
    {u'idx': 10, u'name': u'word_freq_receive'},
    {u'idx': 11, u'name': u'word_freq_will'},
    {u'idx': 12, u'name': u'word_freq_people'},
    {u'idx': 13, u'name': u'word_freq_report'},
    {u'idx': 14, u'name': u'word_freq_addresses'},
    {u'idx': 15, u'name': u'word_freq_free'},
    {u'idx': 16, u'name': u'word_freq_business'},
    {u'idx': 17, u'name': u'word_freq_email'},
    {u'idx': 18, u'name': u'word_freq_you'},
    {u'idx': 19, u'name': u'word_freq_credit'},
    {u'idx': 20, u'name': u'word

In [32]:
from pyspark.ml.feature import ChiSqSelector

In [33]:
cs_selector = ChiSqSelector(numTopFeatures=10, outputCol='selected_features',featuresCol='features').fit(X_all)
X_chisq = cs_selector.transform(X_all)

In [34]:
X_chisq.show(5)

+--------------------+-----+--------------------+
|            features|label|   selected_features|
+--------------------+-----+--------------------+
|(57,[1,2,4,11,15,...|  1.0|(10,[1,2,3],[0.64...|
|(57,[0,1,2,4,5,6,...|  1.0|(10,[0,1,2,3,4,5,...|
|(57,[0,2,4,5,6,7,...|  1.0|(10,[0,2,3,4,5,6,...|
|(57,[4,6,7,8,9,10...|  1.0|(10,[3,5,6,7,8,9]...|
|(57,[4,6,7,8,9,10...|  1.0|(10,[3,5,6,7,8,9]...|
+--------------------+-----+--------------------+
only showing top 5 rows



In [35]:
X_chisq.schema['selected_features'].metadata

{u'ml_attr': {u'attrs': {u'numeric': [{u'idx': 0, u'name': u'word_freq_make'},
    {u'idx': 1, u'name': u'word_freq_address'},
    {u'idx': 2, u'name': u'word_freq_all'},
    {u'idx': 3, u'name': u'word_freq_our'},
    {u'idx': 4, u'name': u'word_freq_over'},
    {u'idx': 5, u'name': u'word_freq_remove'},
    {u'idx': 6, u'name': u'word_freq_internet'},
    {u'idx': 7, u'name': u'word_freq_order'},
    {u'idx': 8, u'name': u'word_freq_mail'},
    {u'idx': 9, u'name': u'word_freq_receive'}]},
  u'num_attrs': 10}}

In [206]:
[input_features[i] for i in cs_selector.selectedFeatures]

['word_freq_make',
 'word_freq_address',
 'word_freq_all',
 'word_freq_our',
 'word_freq_over',
 'word_freq_remove',
 'word_freq_internet',
 'word_freq_order',
 'word_freq_mail',
 'word_freq_receive']

In [40]:
from pyspark.ml.classification import DecisionTreeClassifier

In [207]:
X_all.show(5)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(57,[1,2,4,11,15,...|  1.0|
|(57,[0,1,2,4,5,6,...|  1.0|
|(57,[0,2,4,5,6,7,...|  1.0|
|(57,[4,6,7,8,9,10...|  1.0|
|(57,[4,6,7,8,9,10...|  1.0|
+--------------------+-----+
only showing top 5 rows



In [63]:
assembler = VectorAssembler(inputCols=input_features, outputCol='assembled_features')

In [37]:
len(input_features)

57

In [64]:
feature_selector = ChiSqSelector(featuresCol='assembled_features', outputCol='selected_features')

In [66]:
tree_model = DecisionTreeClassifier(maxDepth=5, featuresCol='selected_features')

In [42]:
from pyspark.ml import Pipeline as SparkPipeline

In [67]:
pipe = SparkPipeline(stages=[assembler, feature_selector, tree_model])

In [68]:
train, test = df.randomSplit([0.8, 0.2])

In [69]:
model = pipe.fit(train)

In [46]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [70]:
BinaryClassificationEvaluator().evaluate(model.transform(test)) #AUC

0.8528091216615807

In [48]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [81]:
param_grid = (ParamGridBuilder()
              .addGrid(tree_model.maxDepth, [2,4,6])
              .addGrid(feature_selector.numTopFeatures, [5, 10])
              .build())

In [82]:
cross_validation = CrossValidator(estimator=pipe,
                                  estimatorParamMaps=param_grid,
                                  evaluator=BinaryClassificationEvaluator(),
                                  numFolds=3)

In [83]:
vc_model = cross_validation.fit(df)

In [84]:
vc_model.avgMetrics

[0.6029003203493974,
 0.40154952633915514,
 0.3668465604187714,
 0.37857501954760564,
 0.35723730720254887,
 0.3139254835323284]

In [92]:
import numpy as np

In [93]:
vc_model.getEstimatorParamMaps()[np.argmax(vc_model.avgMetrics)]

{Param(parent=u'DecisionTreeClassifier_4d6c8ae3ab982daa99b8', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 2,
 Param(parent=u'ChiSqSelector_4d2095dc465107938a6c', name='numTopFeatures', doc='Number of features that selector will select, ordered by ascending p-value. If the number of features is < numTopFeatures, then this will select all features.'): 5}

In [95]:
from pyspark.ml.tuning import TrainValidationSplit

In [96]:
train_test = TrainValidationSplit(estimator=pipe,
                                  estimatorParamMaps=param_grid,
                                  evaluator=BinaryClassificationEvaluator())

In [97]:
tv_model = train_test.fit(df)

In [98]:
tv_model.validationMetrics

[0.6052208698030634,
 0.3882285657449771,
 0.3180528272329421,
 0.24216418838273324,
 0.3324003506067237,
 0.27447098169882633]

# NLP

In [1]:
import wikipedia

In [2]:
obama_page = wikipedia.page("Barack Obama")

In [99]:
import re

In [103]:
match = re.search('president', obama_page.content)

In [104]:
match

In [113]:
re.findall('president', obama_page.content)

[u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president']

In [115]:
re.match('president', obama_page.content) == None

True

In [117]:
re.findall('[Pp]resident', obama_page.content)

[u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'President',
 u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'president',
 u'President',
 u'President',
 u'President',
 u'president',
 u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'President',
 u'president',
 u'president',
 u'President',
 u'President',
 u'President',
 u'president',
 u'president',
 u'president',
 u'president',
 u'president',
 u'President',
 u'President',
 u'President',
 u'President',
 u'president',
 u'President',
 u'President',
 u'President',
 u'president',
 u'President',
 u'president',
 u'President',
 u'President',
 u'President',
 u'President',
 u'President',
 u'President',
 u'president',
 u'President',
 u'President',
 u'President',
 u'preside

In [123]:
re.search('[Pp]resident [A-Za-z]', obama_page.content).group(0)

u'President o'

In [128]:
re.search('[Pp]resident ([A-Za-z]+)', obama_page.content).group(1)

u'of'

In [129]:
re.findall('[Pp]resident ([A-Za-z]+)', obama_page.content)

[u'of',
 u'of',
 u'to',
 u'of',
 u'have',
 u'of',
 u'of',
 u'and',
 u'Bush',
 u'Bush',
 u'of',
 u'of',
 u'he',
 u'Bill',
 u'and',
 u'in',
 u'since',
 u'Obama',
 u'took',
 u'George',
 u'Ronald',
 u'Obama',
 u'to',
 u'to',
 u'mentioned',
 u'Valerie',
 u'Joe',
 u'of',
 u'Obama',
 u'Obama',
 u'Obama',
 u'Joe',
 u'Biden',
 u'to',
 u'Dmitry',
 u'Obama',
 u'Assad',
 u'Obama',
 u'Ra',
 u'Barack',
 u'to',
 u'to',
 u'ever',
 u'to',
 u'Obama',
 u'of',
 u'to',
 u'Barack',
 u'Obama',
 u'Franklin',
 u'Bill',
 u'Obama',
 u'George',
 u'Obama',
 u'Jimmy',
 u'Ronald']

In [146]:
words = [i.lower() for i in obama_page.content.split(' ')]

In [147]:
float(len(set(words))) / len(words)

0.3134826883910387

In [279]:
from nltk.stem.porter import PorterStemmer

In [280]:
stemmer = PorterStemmer()

In [140]:
stemmer.stem('carefully')

u'care'

In [145]:
stemmer.stem('scarred')

u'scar'

In [148]:
float(len(set([stemmer.stem(word) for word in words]))) / len(words)

0.27177189409368635

In [149]:
from nltk.stem.wordnet import WordNetLemmatizer

In [150]:
lemmatizer = WordNetLemmatizer()

In [155]:
lemmatizer.lemmatize('cats')

u'cat'

In [157]:
lemmatizer.lemmatize('were', pos='v')

u'be'

In [164]:
lemmatizer.lemmatize('are', pos='v')

u'be'

In [165]:
import nltk.data 

In [166]:
sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')

In [168]:
for sentence in sent_detector.tokenize(obama_page.content):
    if sentence.count('.') > 1:
        print sentence

He previously served in the U.S. Senate representing Illinois from 2005 to 2008, and in the Illinois State Senate from 1997 to 2004.
Obama represented the 13th District for three terms in the Illinois Senate from 1997 to 2004, when he ran for the U.S. Senate.
In foreign policy, Obama increased U.S. troop levels in Afghanistan, reduced nuclear weapons with the U.S.-Russian New START treaty, and ended military involvement in the Iraq War.
During his second term, Obama promoted greater inclusiveness for LGBT Americans, with his administration filing briefs that urged the Supreme Court to strike down same-sex marriage bans as unconstitutional (United States v. Windsor and Obergefell v. Hodges).
In foreign policy, Obama ordered military intervention in Iraq in response to gains made by ISIL after the 2011 withdrawal from Iraq, continued the process of ending U.S. combat operations in Afghanistan, promoted discussions that led to the 2015 Paris Agreement on global climate change, initiated t

In [169]:
len(sent_detector.tokenize(obama_page.content))

477

In [170]:
float(len(words)) / len(sent_detector.tokenize(obama_page.content))

25.733752620545072

In [1]:
from nltk.probability import ConditionalFreqDist

In [4]:
from nltk import bigrams

In [8]:
list(bigrams(obama_page.content))[:5]

[(u'B', u'a'), (u'a', u'r'), (u'r', u'a'), (u'a', u'c'), (u'c', u'k')]

In [7]:
from nltk.tokenize import word_tokenize

In [9]:
tokenized_text = word_tokenize(obama_page.content)

In [10]:
tokenized_text[:5]

[u'Barack', u'Hussein', u'Obama', u'II', u'(']

In [12]:
list(bigrams(tokenized_text))[:5]

[(u'Barack', u'Hussein'),
 (u'Hussein', u'Obama'),
 (u'Obama', u'II'),
 (u'II', u'('),
 (u'(', u'US')]

In [47]:
lowercase_words = [token.lower() for token in tokenized_text]

In [49]:
cfd = ConditionalFreqDist(bigrams(lowercase_words))

In [50]:
cfd['president']

FreqDist({u"'s": 2,
          u',': 3,
          u'.': 6,
          u'and': 2,
          u'assad': 1,
          u'barack': 2,
          u'biden': 1,
          u'bill': 2,
          u'bush': 2,
          u'dmitry': 1,
          u'ever': 1,
          u'franklin': 1,
          u'george': 2,
          u'have': 1,
          u'he': 1,
          u'in': 1,
          u'jimmy': 1,
          u'joe': 2,
          u'mentioned': 1,
          u'obama': 11,
          u'of': 9,
          u'ra\xfal': 1,
          u'ronald': 2,
          u'since': 1,
          u'to': 8,
          u'took': 1,
          u'valerie': 1})

In [51]:
def gen_text(word, bigram_cdf):
    while True:
        yield word
        word = bigram_cdf[word].max()

In [52]:
from itertools import islice

In [53]:
list(islice(gen_text('president', cfd), 10))

['president',
 u'obama',
 u"'s",
 u'health',
 u'care',
 u'act',
 u'of',
 u'the',
 u'first',
 u'time']

In [54]:
cfd['president']['obama']

11

In [62]:
list(bigrams(lowercase_words)).count(('president', 'obama'))

11

In [57]:
cfd['president']['obama'] / float(lowercase_words.count('president'))

0.16417910447761194

In [63]:
from nltk.probability import ConditionalProbDist, MLEProbDist

In [64]:
cpd = ConditionalProbDist(cfd, MLEProbDist)

In [67]:
p = cpd['president']

In [68]:
p.prob('obama')

0.16417910447761194

In [94]:
def prob_sent(sent, cpd):
    words = [word.lower() for word in word_tokenize(sent)]
    p = 1
    for w1, w2 in zip(words, words[1:]):
        p *= cpd[w1].prob(w2)
    return p

In [95]:
prob_sent('the health care act', cpd)

0.0007120816520294325

In [75]:
p.logprob('obama')

-2.606657571820475

In [76]:
def log_prob_sent(sent, cpd):
    words = [word.lower() for word in word_tokenize(sent)]
    return sum(cpd[w1].logprob(w2) for w1, w2 in zip(words, words[1:]))

In [81]:
log_prob_sent('the health care act', cpd)

-10.4556696998411

In [93]:
2**log_prob_sent('the health care act', cpd)

0.0007120816520294322

In [104]:
from nltk import Text

In [99]:
text = Text(word_tokenize(obama_page.content))

In [100]:
text.concordance('obama')

Displaying 25 of 300 matches:
                                     Obama II ( US /bəˈrɑːk huːˈseɪn oʊˈbɑːmə/ 
ois State Senate from 1997 to 2004 . Obama was born in Honolulu , Hawaii , two 
unity organizer in Chicago . In 1988 Obama enrolled in Harvard Law School , whe
icago Law School from 1992 to 2004 . Obama represented the 13th District for th
 , when he ran for the U.S. Senate . Obama received national attention in 2004 
r election to the Senate . In 2008 , Obama was nominated for president , a year
uary 20 , 2009 . Nine months later , Obama was named the 2009 Nobel Peace Prize
ring his first two years in office , Obama signed many landmark bills . Main re
ebate over the national debt limit , Obama signed the Budget Control and the Am
er Relief Acts . In foreign policy , Obama increased U.S. troop levels in Afgha
ing Republican nominee Mitt Romney , Obama was sworn in for a second term in 20
m in 2013 . During his second term , Obama promoted greater inclusiveness for L
Windsor an

In [102]:
text.similar('conflict')

cia evening continent people successes media compound original
incident moon environment bush year ban house war


In [103]:
text.collocations()

United States; Barack Obama; White House; health care; bin Laden;
general election; Affordable Care; Supreme Court; Harvard Law; Bill
Clinton; New York; Law School; Patient Protection; Democratic Party;
same-sex marriage; Job Creation; Obama announced; National Convention;
Obama signed; high school


In [196]:
n_train = int(len(text)*0.8)

In [197]:
train, test = text[:n_train], text[n_train:]

In [171]:
class BiGramModel(object):
    def __init__(self, train):
        self.cpd = ConditionalProbDist(ConditionalFreqDist(bigrams(word.lower() for word in train)), MLEProbDist)
    def log_prob(self, text):
        return sum(self.cpd[w1].logprob(w2) for w1, w2 in zip(text, text[1:]))
    def perplexity(self, text):
        return (2**self.log_prob(text))**-len(text)

In [172]:
BiGramModel(train).log_prob(test)

-6.260000000000063e+302

In [173]:
BiGramModel(train).cpd['new'].prob('car')

0.0

In [174]:
from nltk.probability import LaplaceProbDist

In [198]:
class LaplaceBiGramModel(object):
    def __init__(self, train):
        self.cpd = ConditionalProbDist(ConditionalFreqDist(bigrams(word.lower() for word in train)),
                                       LaplaceProbDist, len(set(word.lower() for word in train)))
    def log_prob(self, text):
        return sum(self.cpd[w1].logprob(w2) for w1, w2 in zip(text, text[1:]))
    def perplexity(self, text):
        return 2**(-self.log_prob(text)/len(text))

In [199]:
LaplaceNGramModel(train).cpd['new'].prob('car')

0.0003616636528028933

In [200]:
LaplaceBiGramModel(train).perplexity(test)

1937.8534011450295

http://www.cs.cornell.edu/people/pabo/movie-review-data/

In [2]:
import os

In [3]:
positive = [open('data/review_polarity/txt_sentoken/pos/' + text_file).read().split() 
            for text_file in os.listdir('data/review_polarity/txt_sentoken/pos/')]

In [4]:
negative = [open('data/review_polarity/txt_sentoken/neg/' + text_file).read().split() 
            for text_file in os.listdir('data/review_polarity/txt_sentoken/neg/')]

In [5]:
from nltk.probability import FreqDist
from itertools import chain, ifilter

In [6]:
from nltk.corpus import stopwords

In [7]:
en_stop = set(stopwords.words('english'))
en_stop.update(set(',."();:?-!'))

In [8]:
fd = FreqDist(ifilter(lambda x:x not in en_stop, chain(chain(*positive), chain(*negative))))

In [288]:
fd.most_common(50)

[('film', 8849),
 ('one', 5514),
 ('movie', 5429),
 ("it's", 3696),
 ('like', 3543),
 ('even', 2554),
 ('good', 2313),
 ('time', 2280),
 ('story', 2110),
 ('would', 2041),
 ('much', 2022),
 ('also', 1965),
 ('get', 1920),
 ('character', 1902),
 ('two', 1824),
 ('characters', 1813),
 ('first', 1766),
 ('see', 1726),
 ('way', 1668),
 ('well', 1654),
 ('make', 1590),
 ('really', 1556),
 ('films', 1513),
 ('little', 1487),
 ('life', 1467),
 ('plot', 1448),
 ('people', 1418),
 ('could', 1395),
 ('scene', 1372),
 ('bad', 1372),
 ('never', 1360),
 ('best', 1298),
 ('new', 1275),
 ('many', 1267),
 ("doesn't", 1266),
 ('scenes', 1262),
 ('man', 1255),
 ('know', 1207),
 ("don't", 1198),
 ('movies', 1180),
 ("he's", 1149),
 ('great', 1138),
 ('--', 1136),
 ('another', 1111),
 ('love', 1087),
 ('action', 1073),
 ('go', 1073),
 ('us', 1065),
 ('director', 1054),
 ('*', 1054)]

In [9]:
words = [w for w, c in fd.most_common(100)]

In [11]:
import pandas as pd
from sklearn.naive_bayes import BernoulliNB

In [12]:
data_pos = pd.DataFrame({w: [w in set(t) for t in positive] for w in words})

In [13]:
data_pos['label'] = 1

In [14]:
data_neg = pd.DataFrame({w: [w in set(t) for t in negative] for w in words})

In [15]:
data_neg['label'] = 0

In [16]:
data = data_pos.append(data_neg)

In [17]:
from sklearn.model_selection import cross_val_score

In [22]:
cv_scores = cross_val_score(BernoulliNB(), data[words], data.label, cv=10)

In [23]:
cv_scores.mean(), cv_scores.std()

(0.69299999999999995, 0.032031234756093915)

In [298]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [300]:
analyzer = SentimentIntensityAnalyzer()

In [313]:
analyzer.polarity_scores("This is amazing!")

{'compound': 0.6239, 'neg': 0.0, 'neu': 0.328, 'pos': 0.672}

In [318]:
analyzer.polarity_scores("It was a poor performance.")

{'compound': -0.4767, 'neg': 0.508, 'neu': 0.492, 'pos': 0.0}

In [306]:
pos_acc = [analyzer.polarity_scores(" ".join(t))['compound'] > 0 for t in positive]

In [309]:
import numpy as np

In [310]:
np.mean(pos_acc)

0.82799999999999996

In [311]:
neg_acc = [analyzer.polarity_scores(" ".join(t))['compound'] < 0 for t in negative]

In [312]:
np.mean(neg_acc)

0.44500000000000001

In [332]:
dfb = FreqDist(ifilter(lambda x: x[0] not in en_stop and x[1] not in en_stop,
                       chain(bigrams(chain(*positive)), bigrams(chain(*negative)))))

In [333]:
dfb.most_common(20)

[(('special', 'effects'), 374),
 (('*', '*'), 313),
 (('new', 'york'), 243),
 (('even', 'though'), 222),
 (('high', 'school'), 171),
 (('star', 'wars'), 157),
 (('looks', 'like'), 137),
 (('science', 'fiction'), 124),
 (("don't", 'know'), 120),
 (('takes', 'place'), 117),
 (('star', 'trek'), 117),
 (('one', 'thing'), 113),
 (("i've", 'seen'), 112),
 (('running', 'time'), 111),
 (('supporting', 'cast'), 110),
 (('years', 'ago'), 109),
 (('one', 'day'), 109),
 (('ever', 'seen'), 105),
 (('first', 'film'), 102),
 (('look', 'like'), 101)]

In [336]:
mcb = set(i for i, _ in dfb.most_common(200))

In [337]:
data_pos_b = pd.DataFrame({b: [b in set(bigrams(t)) for t in positive] for b in mcb})

In [339]:
data_neg_b = pd.DataFrame({b: [b in set(bigrams(t)) for t in negative] for b in mcb})

In [340]:
dfb.update